In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils as ku
from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [2]:
data = """The cat and her kittens
They put on their mittens,
To eat a Christmas pie.
The poor little kittens
They lost their mittens,
And then they began to cry.
O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens."
"If it be so, ye shall not go,
For ye are naughty kittens."""

In [3]:
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_word = len(tokenizer.word_index)+1
corpus


['the cat and her kittens',
 'they put on their mittens,',
 'to eat a christmas pie.',
 'the poor little kittens',
 'they lost their mittens,',
 'and then they began to cry.',
 'o mother dear, we sadly fear',
 'we cannot go to-day,',
 'for we have lost our mittens."',
 '"if it be so, ye shall not go,',
 'for ye are naughty kittens.']

In [4]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
# print(input_sequences)

In [5]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
predictors, label = input_sequences[:,:-1], input_sequences[:, -1]

In [7]:
label = ku.to_categorical(label, num_classes=total_word)



In [8]:
def create_model(predictor, label, max_sequence_len, total_word):
    input_len = max_sequence_len - 1
    
    model = Sequential()
    model.add(Embedding(total_word, 10, input_length=input_len))
    model.add(LSTM(50))
    model.add(Dense(total_word, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictor, label, epochs=100)
    return model

In [9]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break;
        seed_text += " " + output_word
    return seed_text

In [10]:
model = create_model(predictors, label, max_sequence_len, total_word)
# embedding = model.layers[0].get_weights()[0]
# print(embedding)




Train on 48 samples
Epoch 1/100
48/48 [==============================] - 2s 44ms/sample - loss: 3.7615
Epoch 2/100
48/48 [==============================] - 0s 789us/sample - loss: 3.7591
Epoch 3/100
48/48 [==============================] - 0s 783us/sample - loss: 3.7571
Epoch 4/100
48/48 [==============================] - 0s 639us/sample - loss: 3.7553
Epoch 5/100
48/48 [==============================] - 0s 451us/sample - loss: 3.7537
Epoch 6/100
48/48 [==============================] - 0s 376us/sample - loss: 3.7517
Epoch 7/100
48/48 [==============================] - 0s 425us/sample - loss: 3.7497
Epoch 8/100
48/48 [==============================] - 0s 530us/sample - loss: 3.7477
Epoch 9/100
48/48 [==============================] - 0s 431us/sample - loss: 3.7455
Epoch 10/100
48/48 [==============================] - 0s 725us/sample - loss: 3.7433
Epoch 11/100
48/48 [==============================] - 0s 977us/sample - loss: 3.7405
Epoch 12/100
48/48 [==============================] - 0

48/48 [==============================] - 0s 798us/sample - loss: 2.5376
Epoch 99/100
48/48 [==============================] - 0s 547us/sample - loss: 2.4986
Epoch 100/100
48/48 [==============================] - 0s 1ms/sample - loss: 2.4724


In [11]:
text = generate_text("come here", 5, max_sequence_len, model)
print(text)
# embedding.shape

come here then then to to to
